In [1]:
import pandas as pd

In [12]:
data_df = pd.read_csv('./source/Dataset/cardio_data.csv')

In [13]:
data_df.head()

,date,country,id,active,age,alco,ap_hi,ap_lo,cholesterol,gender,gluc,height,occupation,smoke,weight,disease
0,03-05-2021,Indonesia,0,1,18393,0,110,80,1,2,1,168,Architect,0,62.0,0
1,05-08-2021,Malaysia,1,1,20228,0,140,90,3,1,1,156,Accountant,0,85.0,1
2,13-11-2022,Indonesia,2,0,18857,0,130,70,3,1,1,165,Chef,0,64.0,1
3,31-10-2018,Singapore,3,1,17623,0,150,100,1,2,1,169,Lawyer,0,82.0,1
4,25-09-2020,Singapore,4,0,17474,0,100,60,1,1,1,156,Architect,0,56.0,0


In [14]:
len(data_df)

70000

In [15]:
print(data_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70000 entries, 0 to 69999
Data columns (total 16 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   date         70000 non-null  object 
 1   country      70000 non-null  object 
 2   id           70000 non-null  int64  
 3   active       70000 non-null  int64  
 4   age          70000 non-null  int64  
 5   alco         70000 non-null  int64  
 6   ap_hi        70000 non-null  int64  
 7   ap_lo        70000 non-null  int64  
 8   cholesterol  70000 non-null  int64  
 9   gender       70000 non-null  int64  
 10  gluc         70000 non-null  int64  
 11  height       70000 non-null  int64  
 12  occupation   70000 non-null  object 
 13  smoke        70000 non-null  int64  
 14  weight       70000 non-null  float64
 15  disease      70000 non-null  int64  
dtypes: float64(1), int64(12), object(3)
memory usage: 8.5+ MB
None


In [16]:
for column in data.columns:
    unique_values = data[column].unique()
    print(f"Unique values for column '{column}': {unique_values}")

Unique values for column 'date': ['03-05-2021' '05-08-2021' '13-11-2022' ... '23-01-2020' '09-02-2018'
 '07-11-2021']
Unique values for column 'country': ['Indonesia' 'Malaysia' 'Singapore' 'India']
Unique values for column 'id': [    0     1     2 ... 99996 99998 99999]
Unique values for column 'active': [1 0]
Unique values for column 'age': [18393 20228 18857 ... 14925 17727 17926]
Unique values for column 'alco': [0 1]
Unique values for column 'ap_hi': [  110   140   130   150   100   120   145   170   135   125    90   180
   160   133   190    80   122   169   126   158   200    14   123    70
   161   147   115   137   153    11   148   105   220   119   141   165
   164    12   124   172   902   162   906   117   134   166   210   176
   116    10   121    16   112   159   113   118   155   142   131   157
   136   146   138  -100   909   109    85   106   129    93     7    95
   179   156   168   132   104   103   178   175   128   151    15   139
 11500   127    17   108   14

In [25]:
data_df['age_years'] = data_df['age'] / 365
data_df['age_years'] = data_df['age_years'].astype(int)
data_df.drop(columns=['age'])

,date,country,id,active,alco,ap_hi,ap_lo,cholesterol,gender,gluc,height,occupation,smoke,weight,disease,age_years
0,03-05-2021,Indonesia,0,1,0,110,80,1,2,1,168,Architect,0,62.0,0,50
1,05-08-2021,Malaysia,1,1,0,140,90,3,1,1,156,Accountant,0,85.0,1,55
2,13-11-2022,Indonesia,2,0,0,130,70,3,1,1,165,Chef,0,64.0,1,51
3,31-10-2018,Singapore,3,1,0,150,100,1,2,1,169,Lawyer,0,82.0,1,48
4,25-09-2020,Singapore,4,0,0,100,60,1,1,1,156,Architect,0,56.0,0,47
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69995,03-04-2018,Singapore,99993,1,0,120,80,1,2,1,168,Doctor,1,76.0,0,52
69996,12-01-2022,Malaysia,99995,1,0,140,90,2,1,2,158,Accountant,0,126.0,1,61
69997,25-08-2022,Malaysia,99996,0,1,180,90,3,2,1,183,Accountant,0,105.0,1,52
69998,13-07-2020,Singapore,99998,0,0,135,80,1,1,2,163,Accountant,0,72.0,1,61


In [23]:
data_df

,date,country,id,active,age,alco,ap_hi,ap_lo,cholesterol,gender,gluc,height,occupation,smoke,weight,disease,age_years
0,03-05-2021,Indonesia,0,1,18393,0,110,80,1,2,1,168,Architect,0,62.0,0,50
1,05-08-2021,Malaysia,1,1,20228,0,140,90,3,1,1,156,Accountant,0,85.0,1,55
2,13-11-2022,Indonesia,2,0,18857,0,130,70,3,1,1,165,Chef,0,64.0,1,51
3,31-10-2018,Singapore,3,1,17623,0,150,100,1,2,1,169,Lawyer,0,82.0,1,48
4,25-09-2020,Singapore,4,0,17474,0,100,60,1,1,1,156,Architect,0,56.0,0,47
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69995,03-04-2018,Singapore,99993,1,19240,0,120,80,1,2,1,168,Doctor,1,76.0,0,52
69996,12-01-2022,Malaysia,99995,1,22601,0,140,90,2,1,2,158,Accountant,0,126.0,1,61
69997,25-08-2022,Malaysia,99996,0,19066,1,180,90,3,2,1,183,Accountant,0,105.0,1,52
69998,13-07-2020,Singapore,99998,0,22431,0,135,80,1,1,2,163,Accountant,0,72.0,1,61
